In [4]:
import numpy as np
import torch
from torch.nn import functional as F

from string import ascii_lowercase

from models.model import tokenize, make_weights, embed, batch_tokens, BasicSelfAttention, FeedForwardRelu

In [5]:
input_string = "abacdbee"
output_string = list(set(input_string))

In [6]:
# set implementation in python
def simple_set_dedup(input_string):
  s = set()
  r = []
  for c in input_string:
    if c not in s:
      s.add(c)
      r.append(c)

  return r

In [7]:
simple_set_dedup(input_string)

['a', 'b', 'c', 'd', 'e']

In [8]:
default_tokens = {c: x for x, c in enumerate('abcde')}

In [9]:
default_tokens

{'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4}

In [10]:
tokens, alphabet_size = tokenize(input_string)
tokens, alphabet_size

(array([0, 1, 0, 2, 3, 1, 4, 4]), 26)

In [11]:
batch = batch_tokens(tokens)
batch

array([[0, 1, 0, 2, 3, 1, 4, 4]])

In [12]:
def generate_position_embeddings(batch_size, sequence_length: int):
  M = np.eye(sequence_length)
  return torch.tensor(M).repeat(batch_size, 1, 1)

In [13]:
generate_position_embeddings(2, 3)

tensor([[[1., 0., 0.],
         [0., 1., 0.],
         [0., 0., 1.]],

        [[1., 0., 0.],
         [0., 1., 0.],
         [0., 0., 1.]]], dtype=torch.float64)

In [14]:
embeddings = embed(batch, alphabet_size)
embedding_length = embeddings.shape[2]
embeddings

tensor([[[0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
          0., 0.],
         [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
          0., 0.],
         [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
          0., 0.],
         [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
          0., 0.],
         [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
          0., 0.],
         [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0

In [15]:
embeddings.shape # should be (batches, sequence length, embedding dimension)

torch.Size([1, 8, 36])

In [16]:
raw_weights = torch.bmm(embeddings, embeddings.transpose(1, 2))
raw_weights

tensor([[[2., 0., 1., 0., 0., 0., 0., 0.],
         [0., 2., 0., 0., 0., 1., 0., 0.],
         [1., 0., 2., 0., 0., 0., 0., 0.],
         [0., 0., 0., 2., 0., 0., 0., 0.],
         [0., 0., 0., 0., 2., 0., 0., 0.],
         [0., 1., 0., 0., 0., 2., 0., 0.],
         [0., 0., 0., 0., 0., 0., 2., 1.],
         [0., 0., 0., 0., 0., 0., 1., 2.]]], dtype=torch.float64)

In [17]:
weights = raw_weights #F.softmax(raw_weights, dim=2)
weights

tensor([[[2., 0., 1., 0., 0., 0., 0., 0.],
         [0., 2., 0., 0., 0., 1., 0., 0.],
         [1., 0., 2., 0., 0., 0., 0., 0.],
         [0., 0., 0., 2., 0., 0., 0., 0.],
         [0., 0., 0., 0., 2., 0., 0., 0.],
         [0., 1., 0., 0., 0., 2., 0., 0.],
         [0., 0., 0., 0., 0., 0., 2., 1.],
         [0., 0., 0., 0., 0., 0., 1., 2.]]], dtype=torch.float64)

In [18]:
y = BasicSelfAttention(False).forward(embeddings)
y[0]

tensor([[0., 0., 3., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 3., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 0., 1., 0., 0.],
        [0., 0., 3., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 2., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 0.],
        [0., 0., 0., 3., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 2., 0., 0.],
        [0., 0., 0., 0

In [19]:
attn_out = y[0]
sequence_length = attn_out.shape[0]
sequence_length, alphabet_size

(8, 26)

In [20]:
xx = torch.nn.Linear(embedding_length, embedding_length * 4, False)
yy = torch.nn.Linear(embedding_length * 4, embedding_length, False)

weight_tensor, weight_tensor_2 = make_weights(alphabet_size, sequence_length)


In [21]:
input_string

'abacdbee'

In [23]:
FeedForwardRelu(alphabet_size, sequence_length)(y)

TypeError: cannot assign 'torch.FloatTensor' as parameter 'weight' (torch.nn.Parameter or None expected)

In [21]:
y

tensor([[[0., 0., 3., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 0., 1., 0., 0., 0.,
          0., 0.],
         [0., 0., 0., 3., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 0., 1.,
          0., 0.],
         [0., 0., 3., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 2., 0., 0., 0.,
          0., 0.],
         [0., 0., 0., 0., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 0., 0.,
          0., 0.],
         [0., 0., 0., 0., 0., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 0.,
          0., 0.],
         [0., 0., 0., 3., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0